<a href="https://colab.research.google.com/github/SarahSchilling618/project-4/blob/julius/PostgreSQL_Analysis_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install psycopg2-binary sqlalchemy

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/25/1f/7ae31759142999a8d06b3e250c1346c4abcdcada8fa884376775dc1de686/psycopg2_binary-2.9.9-cp311-cp311-win_amd64.whl.metadata
  Using cached psycopg2_binary-2.9.9-cp311-cp311-win_amd64.whl.metadata (4.6 kB)
Using cached psycopg2_binary-2.9.9-cp311-cp311-win_amd64.whl (1.2 MB)


In [2]:
import psycopg2
import pandas as pd

# Database connection parameters
user = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'
dbname = 'Dementia'

# Create a connection to the PostgreSQL database
conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

# Create a cursor object
cur = conn.cursor()

# Define a query to get data from a table
query = "SELECT * FROM dementia"

# Execute the query
cur.execute(query)

# Fetch all rows from the executed query
rows = cur.fetchall()

# Convert the query result to a pandas DataFrame
df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])

# Print the DataFrame
pd.set_option('display.max_columns', None)
print(df.head())

# Close the cursor and connection
cur.close()
conn.close()


   Diabetic  AlcoholLevel  HeartRate  BloodOxygenLevel  BodyTemperature  \
0         1      0.084974         98         96.230743        36.224852   
1         0      0.016973         78         93.032122        36.183874   
2         0      0.009000         89         93.566504        37.326321   
3         0      0.086437         60         93.906510        37.030620   
4         1      0.150747         67         97.508994        36.062121   

      Weight  MRI_Delay Prescription  Dosage_in_mg  Age   Education_Level  \
0  57.563978  36.421028         None           NaN   60    Primary School   
1  56.832335  31.157633  Galantamine          12.0   61  Secondary School   
2  59.759066  37.640435         None           NaN   69    Primary School   
3  58.266471  50.673992    Donepezil          23.0   78  Secondary School   
4  67.705027  27.810601    Memantine          20.0   77  Secondary School   

  Dominant_Hand  Gender Family_History  Smoking_Status    APOE_4  \
0          Left  F

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Drop multiple columns using a list
columns_to_drop = ["Prescription", "Dosage_in_mg", "Education_Level", "Dominant_Hand",
                   "Gender", "Family_History", "Smoking_Status", "APOE_4",
                   "Physical_Activity", "Depression_Status", "Medication_History",
                   "Nutrition_Diet", "Sleep_Quality", "Chronic_Health_Conditions"]

df.drop(columns_to_drop, axis=1, inplace=True)

# Prepare the data
X = df.drop("Dementia", axis=1)
y = df["Dementia"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
print("Model Coefficients:", model.coef_)
print("Model Intercept:", model.intercept_)
print("Model Score:", model.score(X_test, y_test))



Model Coefficients: [ 0.0036735  -0.17571744  0.00126677 -0.00411813  0.02597953 -0.00061725
 -0.00076451 -0.00031451 -0.13352848]
Model Intercept: 0.7817313411024698
Model Score: 0.694760404034539


In [6]:
dummies_df = pd.get_dummies(df)
dummies_df

,Diabetic,AlcoholLevel,HeartRate,BloodOxygenLevel,BodyTemperature,Weight,MRI_Delay,Age,Cognitive_Test_Scores,Dementia
0,1,0.084974,98,96.230743,36.224852,57.563978,36.421028,60,10,0
1,0,0.016973,78,93.032122,36.183874,56.832335,31.157633,61,1,1
2,0,0.009000,89,93.566504,37.326321,59.759066,37.640435,69,8,0
3,0,0.086437,60,93.906510,37.030620,58.266471,50.673992,78,5,1
4,1,0.150747,67,97.508994,36.062121,67.705027,27.810601,77,0,1
...,...,...,...,...,...,...,...,...,...,...
995,1,0.081825,87,93.851963,36.495134,50.380106,42.318663,88,5,1
996,1,0.145249,97,94.522391,36.270804,94.006484,52.812568,80,9,0
997,1,0.073692,65,98.578388,37.065703,80.088613,13.640229,67,8,0
998,0,0.037347,71,91.298580,37.037202,95.322210,17.445715,62,2,1


In [7]:
dummies_df.columns

Index(['Diabetic', 'AlcoholLevel', 'HeartRate', 'BloodOxygenLevel',
       'BodyTemperature', 'Weight', 'MRI_Delay', 'Age',
       'Cognitive_Test_Scores', 'Dementia'],
      dtype='object')